<a href="https://colab.research.google.com/github/stakunlena/ich_python_for_DA_HW/blob/main/Stakun_HW_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Домашняя работа №12**

Анализ данных с помощью объединений и конкатенации

Набор данных: Используйте набор данных "tips" и набор данных "flights" из Seaborn.

Инструкции:

- Загрузите наборы данных "tips" и "flights" в отдельные Polars DataFrames.
- Из набора данных 'flights' выберите столбцы 'year', 'month' и 'passengers'.
- Создайте новый DataFrame из 'flights', который агрегирует общее количество пассажиров за год.
- Объедините эти агрегированные данные с набором данных 'tips' по столбцу 'year' (примечание: для этого вам может понадобиться создать столбец 'year' в 'tips').
- Конкатенируйте DataFrame 'tips' (отфильтрованный по времени 'Dinner') с первыми 50 строками 'flights' (подумайте, каким образом).
- Отобразите первые 10 строк объединенного и конкатенированного DataFrame.

**1. Загрузка наборов данных**

In [ ]:
import seaborn as sns
import polars as pl

# 1. Загружаем датасеты из Seaborn (в виде pandas DataFrame)
tips_pd = sns.load_dataset("tips")
flights_pd = sns.load_dataset("flights")

# 2. Преобразуем их в Polars DataFrame
tips = pl.from_pandas(tips_pd)
flights = pl.from_pandas(flights_pd)

# 3. Проверяем результат
print("DataFrame tips:")
print(tips.head())

print("\nDataFrame flights:")
print(flights.head())

# 4. Смотрим схемы (названия и типы столбцов)
print("\nSchema tips:")
print(tips.schema)

print("\nSchema flights:")
print(flights.schema)

DataFrame tips:
shape: (5, 7)
┌────────────┬──────┬────────┬────────┬─────┬────────┬──────┐
│ total_bill ┆ tip  ┆ sex    ┆ smoker ┆ day ┆ time   ┆ size │
│ ---        ┆ ---  ┆ ---    ┆ ---    ┆ --- ┆ ---    ┆ ---  │
│ f64        ┆ f64  ┆ cat    ┆ cat    ┆ cat ┆ cat    ┆ i64  │
╞════════════╪══════╪════════╪════════╪═════╪════════╪══════╡
│ 16.99      ┆ 1.01 ┆ Female ┆ No     ┆ Sun ┆ Dinner ┆ 2    │
│ 10.34      ┆ 1.66 ┆ Male   ┆ No     ┆ Sun ┆ Dinner ┆ 3    │
│ 21.01      ┆ 3.5  ┆ Male   ┆ No     ┆ Sun ┆ Dinner ┆ 3    │
│ 23.68      ┆ 3.31 ┆ Male   ┆ No     ┆ Sun ┆ Dinner ┆ 2    │
│ 24.59      ┆ 3.61 ┆ Female ┆ No     ┆ Sun ┆ Dinner ┆ 4    │
└────────────┴──────┴────────┴────────┴─────┴────────┴──────┘

DataFrame flights:
shape: (5, 3)
┌──────┬───────┬────────────┐
│ year ┆ month ┆ passengers │
│ ---  ┆ ---   ┆ ---        │
│ i64  ┆ cat   ┆ i64        │
╞══════╪═══════╪════════════╡
│ 1949 ┆ Jan   ┆ 112        │
│ 1949 ┆ Feb   ┆ 118        │
│ 1949 ┆ Mar   ┆ 132        │
│ 1949 ┆ Apr  

**2. Выбераем столбцы 'year', 'month' и 'passengers' из набора 'flights'.**

In [ ]:
# 1. Выбираем нужные столбцы: year, month, passengers
flights_selected = flights.select(["year", "month", "passengers"])

# 2. Проверяем результат
print(flights_selected.head())

shape: (5, 3)
┌──────┬───────┬────────────┐
│ year ┆ month ┆ passengers │
│ ---  ┆ ---   ┆ ---        │
│ i64  ┆ cat   ┆ i64        │
╞══════╪═══════╪════════════╡
│ 1949 ┆ Jan   ┆ 112        │
│ 1949 ┆ Feb   ┆ 118        │
│ 1949 ┆ Mar   ┆ 132        │
│ 1949 ┆ Apr   ┆ 129        │
│ 1949 ┆ May   ┆ 121        │
└──────┴───────┴────────────┘


**3. Создаем новый DataFrame из 'flights', который агрегирует общее количество пассажиров за год.**

In [ ]:
# 1. Агрегируем общее количество пассажиров по каждому году
flights_yearly = (
    flights
    .group_by("year")
    .agg(
        pl.col("passengers").sum().alias("total_passengers")
    )
    .sort("year")  # сортируем по возрастанию года
)

# 2. Проверяем результат
print(flights_yearly)

shape: (12, 2)
┌──────┬──────────────────┐
│ year ┆ total_passengers │
│ ---  ┆ ---              │
│ i64  ┆ i64              │
╞══════╪══════════════════╡
│ 1949 ┆ 1520             │
│ 1950 ┆ 1676             │
│ 1951 ┆ 2042             │
│ 1952 ┆ 2364             │
│ 1953 ┆ 2700             │
│ …    ┆ …                │
│ 1956 ┆ 3939             │
│ 1957 ┆ 4421             │
│ 1958 ┆ 4572             │
│ 1959 ┆ 5140             │
│ 1960 ┆ 5714             │
└──────┴──────────────────┘


**4. Объединем агрегированные данные с набором данных 'tips' по столбцу 'year' (примечание: для этого создаем столбец 'year' в 'tips').**

In [ ]:
# 1. Агрегируем общее количество пассажиров по каждому году (через оконную функцию)
flights_yearly = (
    flights
    .with_columns(
        pl.col("passengers").sum().over("year").alias("total_passengers")
    )
    .unique(subset=["year"])   # оставляем по одному ряду на год
)

# 2. Добавляем столбец 'year' в tips (пусть у всех записей будет одинаковый год, например 1950)
tips_with_year = tips.with_columns(
    pl.lit(1950).alias("year")    # создаем константный столбец
)

# 3. Объединяем данные по столбцу 'year'
merged = tips_with_year.join(flights_yearly, on="year", how="left")

# 4. Проверяем результат
print(merged.head(10))
print("\nSchema объединенного DataFrame:")
print(merged.schema)

shape: (10, 11)
┌────────────┬──────┬────────┬────────┬───┬──────┬───────┬────────────┬──────────────────┐
│ total_bill ┆ tip  ┆ sex    ┆ smoker ┆ … ┆ year ┆ month ┆ passengers ┆ total_passengers │
│ ---        ┆ ---  ┆ ---    ┆ ---    ┆   ┆ ---  ┆ ---   ┆ ---        ┆ ---              │
│ f64        ┆ f64  ┆ cat    ┆ cat    ┆   ┆ i32  ┆ cat   ┆ i64        ┆ i64              │
╞════════════╪══════╪════════╪════════╪═══╪══════╪═══════╪════════════╪══════════════════╡
│ 16.99      ┆ 1.01 ┆ Female ┆ No     ┆ … ┆ 1950 ┆ Jan   ┆ 115        ┆ 1676             │
│ 10.34      ┆ 1.66 ┆ Male   ┆ No     ┆ … ┆ 1950 ┆ Jan   ┆ 115        ┆ 1676             │
│ 21.01      ┆ 3.5  ┆ Male   ┆ No     ┆ … ┆ 1950 ┆ Jan   ┆ 115        ┆ 1676             │
│ 23.68      ┆ 3.31 ┆ Male   ┆ No     ┆ … ┆ 1950 ┆ Jan   ┆ 115        ┆ 1676             │
│ 24.59      ┆ 3.61 ┆ Female ┆ No     ┆ … ┆ 1950 ┆ Jan   ┆ 115        ┆ 1676             │
│ 25.29      ┆ 4.71 ┆ Male   ┆ No     ┆ … ┆ 1950 ┆ Jan   ┆ 115        ┆ 16

**5. Конкатенируем DataFrame 'tips' (отфильтрованный по времени 'Dinner') с первыми 50 строками 'flights' диагональной конкатенацией.**

In [ ]:
# 1. Отфильтруем 'tips' по времени 'Dinner'
tips_dinner = tips.filter(pl.col("time") == "Dinner")

# 2. Возьмем первые 50 строк из flights
flights_50 = flights.head(50)

# 3. Приведем к общей структуре (добавим отсутствующие столбцы при необходимости)
# Polars автоматически создаст недостающие столбцы как null при конкатенации
concatenated = pl.concat([tips_dinner, flights_50], how="diagonal")

/tmp/ipython-input-2498731785.py:9: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  concatenated = pl.concat([tips_dinner, flights_50], how="diagonal")


**6. Отображаем первые 10 строк объединенного и конкатенированного DataFrame.**

In [ ]:
print(concatenated.head(10))
print("\nРазмер объединенного DataFrame:", concatenated.shape)

shape: (10, 10)
┌────────────┬──────┬────────┬────────┬───┬──────┬──────┬───────┬────────────┐
│ total_bill ┆ tip  ┆ sex    ┆ smoker ┆ … ┆ size ┆ year ┆ month ┆ passengers │
│ ---        ┆ ---  ┆ ---    ┆ ---    ┆   ┆ ---  ┆ ---  ┆ ---   ┆ ---        │
│ f64        ┆ f64  ┆ cat    ┆ cat    ┆   ┆ i64  ┆ i64  ┆ cat   ┆ i64        │
╞════════════╪══════╪════════╪════════╪═══╪══════╪══════╪═══════╪════════════╡
│ 16.99      ┆ 1.01 ┆ Female ┆ No     ┆ … ┆ 2    ┆ null ┆ null  ┆ null       │
│ 10.34      ┆ 1.66 ┆ Male   ┆ No     ┆ … ┆ 3    ┆ null ┆ null  ┆ null       │
│ 21.01      ┆ 3.5  ┆ Male   ┆ No     ┆ … ┆ 3    ┆ null ┆ null  ┆ null       │
│ 23.68      ┆ 3.31 ┆ Male   ┆ No     ┆ … ┆ 2    ┆ null ┆ null  ┆ null       │
│ 24.59      ┆ 3.61 ┆ Female ┆ No     ┆ … ┆ 4    ┆ null ┆ null  ┆ null       │
│ 25.29      ┆ 4.71 ┆ Male   ┆ No     ┆ … ┆ 4    ┆ null ┆ null  ┆ null       │
│ 8.77       ┆ 2.0  ┆ Male   ┆ No     ┆ … ┆ 2    ┆ null ┆ null  ┆ null       │
│ 26.88      ┆ 3.12 ┆ Male   ┆ No   